# SQL

## Initialisation

Run the [Blue Brain Nexus project creation notebook](https://github.com/BlueBrain/nexus-forge/blob/master/examples/notebooks/nexus-demo/00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/v1) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
#Let get some SHACL shapes from https://github.com/INCF/neuroshapes.git
import os 

neuroshapes_dir = "./neuroshapes"
if os.path.exists(neuroshapes_dir):
  !rm -Rf $neuroshapes_dir
! git clone https://github.com/INCF/neuroshapes.git
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/dataset" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/activity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/entity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/ontology" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/person" "./neuroshapes/shapes/neurosciencegraph/commons/" 

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

## Imports

In [ ]:
!pip install sqlalchemy

In [ ]:
from kgforge.core import Resource

In [ ]:
import pandas as pd

In [ ]:
from sqlalchemy import create_engine

## SQL Database initialization

In [ ]:
engine = create_engine("sqlite:///:memory:")

## Resources to SQL

In [ ]:
jane = Resource(name="Jane Doe", email="jane.doe@epfl.ch")

In [ ]:
john = Resource(name="John Smith", email="john.smith@epfl.ch")

In [ ]:
other = Resource(name="John Doe", email="john.doe@unige.ch")

In [ ]:
persons = [jane, john, other]

Converting Resources to DataFrame

In [ ]:
df = forge.as_dataframe(persons)

Persisting Resources in the SQL Database

NB: The more powerful way is to have a Store implementation for the SQL Database

In [ ]:
df.to_sql("Persons", engine, index=False)

## SQL to Resources

Getting the table as a DataFrame

In [ ]:
pd.read_sql("SELECT * FROM Persons", engine)

Filtering on the SQL Database to keep only persons with an EPFL email

In [ ]:
df = pd.read_sql("SELECT * FROM Persons WHERE email LIKE '%@epfl.ch'", engine)

In [ ]:
df

Converting DataFrame to Resources

In [ ]:
persons = forge.from_dataframe(df)

In [ ]:
print(*persons, sep="\n")

In [ ]:
forge.register(persons)

In [ ]:
persons[0]._synchronized